### CONFIG

In [1]:
import pdfplumber
import re
import json
import os
import PyPDF2
import re
import pandas as pd
import sys
import os


from dotenv import load_dotenv
from neo4j import GraphDatabase
from py2neo import Graph, Node, Relationship
from langchain_openai import AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import List, Dict, Tuple

In [37]:
load_dotenv()
# Recupera endpoint e chiave dalle variabili d'ambiente
api_base = os.getenv("AZURE_OPENAI_API_BASE")
api_key = os.getenv("AZURE_OPENAI_API_KEY")

# neo4j first instance parameters
NEO4J_URI= "neo4j+s://0482640f.databases.neo4j.io"
NEO4J_USERNAME= "neo4j"
NEO4J_PASSWORD= "PNvdaZlk326-ja2hRD1K97ZUUMnD4mj0NsecZNu5-9k"
AURA_INSTANCEID= "0482640f"
AURA_INSTANCENAME= "Instance01"

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

In [3]:
def crea_llm(api_key: str, api_base: str) -> AzureChatOpenAI:
    """
    Crea un'istanza del modello linguistico Azure OpenAI.
    
    Args:
        api_key (str): La chiave API di Azure
        api_base (str): L'URL base dell'API Azure
        
    Returns:
        AzureChatOpenAI: Istanza configurata del modello
    """
    return AzureChatOpenAI(
        openai_api_version="2024-08-01-preview",
        azure_deployment="o1-mini",
        azure_endpoint=api_base,
        api_key=api_key,
        temperature=1
    )

llm = crea_llm(api_key, api_base)

### METHODS

In [4]:
# ESTRAGGO LO SCHEMA in modo da aiutare LLM a generare le domande

def get_database_schema(driver) -> str:
    """Estrae automaticamente lo schema del database, incluse le proprietà dei nodi e delle relazioni."""
    with driver.session() as session:
        # Query per ottenere i tipi di nodi
        node_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels
        RETURN COLLECT(labels) AS types
        """
        
        # Query per ottenere i tipi di relazioni e la loro struttura
        rel_query = """
        MATCH (a)-[r]->(b)
        WITH DISTINCT labels(a) AS sourceLabels, type(r) AS relType, labels(b) AS targetLabels
        RETURN sourceLabels, relType, targetLabels
        """
        
        # Query per ottenere le proprietà dei nodi
        node_properties_query = """
        MATCH (n)
        WITH DISTINCT labels(n) AS labels, keys(n) AS properties
        UNWIND properties AS property
        RETURN labels, property
        """
        
        # Query per ottenere le proprietà delle relazioni
        rel_properties_query = """
        MATCH ()-[r]->()
        WITH DISTINCT type(r) AS relType, keys(r) AS properties
        UNWIND properties AS property
        RETURN relType, property
        """
        
        # Esegui le query separatamente
        node_types = session.run(node_query).single()['types']
        relationships = list(session.run(rel_query))
        
        # Ottieni le proprietà dei nodi
        node_properties = list(session.run(node_properties_query))
        
        # Ottieni le proprietà delle relazioni
        rel_properties = list(session.run(rel_properties_query))
        
        # Costruisci una descrizione dettagliata dello schema
        schema_description = "Schema del database:\n\nNodi:\n"
        for node_type in node_types:
            schema_description += f"- {':'.join(node_type)}\n"
            # Aggiungi le proprietà dei nodi
            for node_property in node_properties:
                if set(node_type).intersection(set(node_property['labels'])):
                    schema_description += f"    Proprietà: {node_property['property']}\n"
        
        schema_description += "\nRelazioni e loro struttura:\n"
        for rel in relationships:
            source = ':'.join(rel['sourceLabels'])
            target = ':'.join(rel['targetLabels'])
            schema_description += f"- ({source})-[:{rel['relType']}]->({target})\n"
            # Aggiungi le proprietà delle relazioni
            for rel_property in rel_properties:
                if rel_property['relType'] == rel['relType']:
                    schema_description += f"    Proprietà: {rel_property['property']}\n"
            
        return schema_description

In [5]:
# LLM genera la domanda basandosi sullo schema
def generate_cypher_query(llm, schema: str, question: str) -> str:
    """Genera una query Cypher basata sulla domanda dell'utente."""
    prompt = ChatPromptTemplate.from_template("""    
    Sei un esperto di Neo4j. Converti la seguente domanda in una query Cypher.
    
    {schema}
    
    Domanda: {question}
    
    Genera SOLO la query Cypher, senza backticks, senza la parola 'cypher' e senza commenti.
    La query deve usare ESATTAMENTE i nomi delle relazioni come appaiono nello schema.
    IMPORTANTE: Usa SOLO proprietà che sicuramente esistono nello schema. Non inventare proprietà come 'quantità' o 'legale_limite' a meno che non siano esplicitamente menzionate nello schema.
    Assicurati di usare la proprietà 'nome' per fare match sui nomi di nodi. 
    Voglio che ritorni solo il 'numero' del piatto, p.numero                                              
    """)
    
    messages = prompt.format_messages(
        schema=schema,
        question=question
    )
    response = llm.invoke(messages, temperature=1.0, seed=42)

    # Pulizia della risposta
    query = response.content.strip()
    query = query.replace('```cypher', '').replace('```', '').strip()
    return query


In [13]:
def generate_cypher_query(llm, schema: str, question: str) -> str:
    """Genera una query Cypher basata sulla domanda dell'utente."""
    
    # Pre-elaborazione per identificare termini fuori contesto
    terms_to_ignore = ["della galassia", "Codice Galattico", "Sirius Cosmo"]  # aggiungi altri termini se necessario
    
    prompt = ChatPromptTemplate.from_template("""
        Sei un esperto di Neo4j. Converti la seguente domanda in una query Cypher.
        
        {schema}
        
        Domanda: {question}
        
        ISTRUZIONI IMPORTANTI:
        - Genera SOLO la query Cypher, senza backticks, senza la parola 'cypher' e senza commenti.
        - La query deve usare ESATTAMENTE i nomi delle relazioni come appaiono nello schema.
        - Usa SOLO proprietà che sicuramente esistono nello schema. Non inventare proprietà.
        - Assicurati di usare la proprietà 'nome' per fare match sui nomi di nodi.
        - Voglio che ritorni solo il 'numero' del piatto, p.numero
        - Ignora i riferimenti a {terms_to_ignore} e concentrati sul resto della domanda.
        - Traduci Sashimi di Magikarp con Sashimi di Magicarp
        - Attento a non includere gli aggettivi nella ricerca delle Sostanze e delle Tecniche. Le Sostanze e le Tecniche hanno sempre la lettera capitale.
        - Se la domanda è completamente fuori contesto, genera una query semplice che restituisca un sottoinsieme di dati rilevanti dallo schema.
    """)
    
    messages = prompt.format_messages(
        schema=schema,
        question=question,
        terms_to_ignore=", ".join(terms_to_ignore)
    )
    
    response = llm.invoke(messages, temperature=1.0, seed=42)
    
    # Pulizia della risposta
    query = response.content.strip()
    query = query.replace('```cypher', '').replace('```', '').strip()
    
    return query

In [14]:
# esegue la query
def execute_query(driver, query: str) -> List[Dict]:
    """Esegue la query Cypher e restituisce i risultati grezzi."""
    with driver.session() as session:
        try:
            result = session.run(query)
            print(f"Risultati grezzi della query: {result}")
            
            results = []
            i = 0 
            for record in result:
                print(f"Record trovato: {record}")
                
                # Gestisci solo i numeri dei piatti (senza altre informazioni)
                if 'p' in record:
                    node_data = record['p']
                    if isinstance(node_data, dict):
                        # Aggiungi solo il numero del piatto
                        if 'numero' in node_data:
                            results.append({node_data['numero']})
                    else:
                        # Aggiungi solo il numero se è un singolo valore
                        results.append({str(node_data)})

                elif any(k.endswith('.numero') for k in record.keys()):
                    # Se la proprietà è '.numero', estrai il numero
                    nome_key = next(k for k in record.keys() if k.endswith('.numero'))
                    results.append({record[nome_key]})
                
                # pigliamo al massimo 20 piatti
                i = i+1
                if i > 20:
                    break

            return results
        except Exception as e:
            print(f"Errore nell'esecuzione della query: {str(e)}")
            return [{'error': str(e)}]

In [15]:
# FUNZIONE DI WRAP! Prima estrae lo schema 
def query_database(driver, llm, schema, question: str) -> dict:
    """Funzione principale che gestisce l'intero processo di query."""
    try:
        # Ottiene lo schema del database
        #schema = get_database_schema(driver)
        #print(f"Schema rilevato:\n{schema}")  # Debug
        
        # Genera la query Cypher
        cypher_query = generate_cypher_query(llm, schema, question)
        print(f"Query Cypher generata: {cypher_query}")  # Debug
        
        # Esegue la query
        results = execute_query(driver, cypher_query)
        
        # Stampa i risultati grezzi come parte del log
        print(f"Risultati grezzi (dati raw): {results}")  # Debug
        
        # Se non ci sono risultati
        if not results:
            return {
                "risultati_grezzi": []
            }
        
        # Restituisce solo i risultati grezzi (numeri dei piatti)
        return {
            "risultati_grezzi": results
        }
        
    except Exception as e:
        # Restituzione di un dizionario anche in caso di errore
        return {
            "risultati_grezzi": [{'error': str(e)}]
        }


In [16]:
schema = get_database_schema(driver)
print(schema)

Schema del database:

Nodi:
- Tecnica
    Proprietà: nome
    Proprietà: nome
    Proprietà: Gravità_Psionica
    Proprietà: nome
    Proprietà: Livello
    Proprietà: nome
    Proprietà: tipo
    Proprietà: caratteristica
    Proprietà: metabolismo
    Proprietà: nome
    Proprietà: version
    Proprietà: nome
    Proprietà: livello
    Proprietà: nome
    Proprietà: livello
    Proprietà: Livello
    Proprietà: nome
    Proprietà: uso
    Proprietà: applicazione
    Proprietà: nome
    Proprietà: uso
    Proprietà: effetto
    Proprietà: nome
    Proprietà: Termocinetica_Multipla
    Proprietà: nome
    Proprietà: Frugale_Energeticamente_Negativa
    Proprietà: nome
    Proprietà: Quantum_Fluttuante
    Proprietà: nome
    Proprietà: Flusso
    Proprietà: nome
    Proprietà: Frugale
    Proprietà: nome
    Proprietà: Dinamico
    Proprietà: nome
    Proprietà: ElettroOsmotica
    Proprietà: nome
    Proprietà: Metodo
    Proprietà: nome
    Proprietà: cabina
    Proprietà: nome
    P

In [17]:
import csv

def read_input_csv(input_file):
    """Legge il file CSV di input e restituisce una lista di domande."""
    with open(input_file, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        questions = [row[0] for row in reader]  # Supponiamo che le domande siano nella prima colonna
    return questions

def write_output_csv(output_file, results):
    """Scrive i risultati nel file CSV di output."""
    with open(output_file, mode='w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["row_id", "result"])  # Intestazione del CSV
        for row_id, result in results:
            writer.writerow([row_id, result])

def query_multiple_questions(driver, llm, input_file, output_file, schema):
    """Esegue più domande e salva i risultati in un CSV."""
    # Leggi le domande dal file di input
    questions = read_input_csv(input_file)
    
    # Prepara una lista per i risultati
    results = []
    
    # Esegui la funzione query_database per ogni domanda
    for row_id, question in enumerate(questions, start=1):
        print(f"\nEseguendo la domanda {row_id}: {question}")
        
        # Esegui la query
        risposta = query_database(driver, llm, schema, question)
        
        # Prendi la risposta naturale (risultato finale con i numeri dei piatti)
        result = risposta["risultati_grezzi"]
        
        # Aggiungi il risultato alla lista dei risultati
        results.append((row_id, result))
    
    # Scrivi i risultati nel file CSV di output
    write_output_csv(output_file, results)

# Esempio di utilizzo
input_file = "C:/Users/lparigi/Downloads/letspizza/Hackapizza Dataset/domande.csv"  # Il tuo file CSV di input contenente le domande
output_file = "risultati.csv"  # Il file CSV di output dove scrivere i risultati

query_multiple_questions(driver, llm, input_file, output_file, schema)



Eseguendo la domanda 1: domanda
Query Cypher generata: MATCH (p:Piatto) RETURN p.numero
Risultati grezzi della query: <neo4j._sync.work.result.Result object at 0x000002317A3DC140>
Record trovato: <Record p.numero=217>
Record trovato: <Record p.numero=None>
Record trovato: <Record p.numero=241>
Record trovato: <Record p.numero=20>
Record trovato: <Record p.numero=59>
Record trovato: <Record p.numero=247>
Record trovato: <Record p.numero=259>
Record trovato: <Record p.numero=265>
Record trovato: <Record p.numero=286>
Record trovato: <Record p.numero=193>
Record trovato: <Record p.numero=135>
Record trovato: <Record p.numero=243>
Record trovato: <Record p.numero=242>
Record trovato: <Record p.numero=179>
Record trovato: <Record p.numero=94>
Record trovato: <Record p.numero=7>
Record trovato: <Record p.numero=78>
Record trovato: <Record p.numero=156>
Record trovato: <Record p.numero=123>
Record trovato: <Record p.numero=137>
Record trovato: <Record p.numero=257>
Risultati grezzi (dati raw

In [18]:
def convert_set_to_comma_string(cell):
    """
    Converte una stringa che rappresenta un insieme di numeri in una stringa di numeri separati da virgole.
    Se la cella è vuota, restituisce "1".
    """
    if not cell or cell == "[]":
        return "\"1\""  # Restituisce "1" per liste vuote con virgolette
    
    # Estrai i numeri utilizzando un'espressione regolare
    numbers = re.findall(r'\{(\d+|\w+)\}', cell)
    
    # Filtra i valori None
    numbers = [num for num in numbers if num != "None"]
    
    # Restituisci i numeri come stringa separata da virgole, racchiusa tra virgolette
    return f"\"{','.join(numbers)}\""

def process_csv(input_path, output_path=None):
    """Processa un file CSV dato il percorso."""
    # Verificare se il file esiste
    if not os.path.exists(input_path):
        print(f"Errore: Il file '{input_path}' non esiste.")
        return
    
    try:
        # Carica il CSV
        df = pd.read_csv(input_path)
        
        # Converti la colonna 'result'
        if 'result' in df.columns:
            df['result'] = df['result'].apply(convert_set_to_comma_string)
            
            # Poiché abbiamo già aggiunto le virgolette manualmente, usiamo quoting=0 (nessun quoting)
            # Creiamo l'output manualmente per evitare problemi con le virgolette
            output_lines = ["row_id,result"]
            for index, row in df.iterrows():
                output_lines.append(f"{row['row_id']},{row['result']}")
            
            output_text = "\n".join(output_lines)
            
            # Salva il CSV risultante se è specificato un percorso di output
            if output_path:
                with open(output_path, 'w') as f:
                    f.write(output_text)
                print(f"File convertito salvato in: {output_path}")
            
            # Mostra il risultato
            print("\nContenuto del CSV convertito:")
            print(output_text)
        else:
            print("Errore: La colonna 'result' non è presente nel CSV.")
    
    except Exception as e:
        print(f"Errore durante l'elaborazione del file: {str(e)}")

input_path = "risultati.csv"
output_path = "risultati_converted.csv"

process_csv(input_path, output_path)

File convertito salvato in: risultati_converted.csv

Contenuto del CSV convertito:
row_id,result
1,"217,241,20,59,247,259,265,286,193,135,243,242,179,94,7,78,156,123,137,257"
2,"78"
3,"225"
4,"156"
5,"215"
6,"94"
7,"179"
8,"189,267,171"
9,"124,6,13,51"
10,"71"
11,"115,184"
12,"104"
13,"185,240"
14,"1"
15,"1"
16,"35,10,160,191,281,147,269"
17,"35,10,160,191,45,281,147"
18,"276,16,157,263,112"
19,"91,37,128"
20,"1"
21,"54"
22,"1"
23,"1"
24,"149"
25,"1"
26,"1"
27,"20"
28,"243,279,21,187,115,92"
29,"243,96,62,205,4"
30,"234"
31,"77,110,112,190"
32,"221,166"
33,"124,128,206"
34,"1"
35,"1"
36,"121"
37,"247,276,72"
38,"1"
39,"1"
40,""
41,"235,139,196,41,158,153,154,49"
42,"191"
43,"179,48"
44,"59,265,123,64,275,37,207"
45,"142,61,169,147,113"
46,""
47,"1"
48,"1"
49,"169"
50,"1"
51,"146"
52,"102,231"
53,"1"
54,"258"
55,"52"
56,"1"
57,"114,272"
58,"1"
59,"1"
60,"1"
61,"1"
62,"94,78,270"
63,"46,77"
64,"1"
65,"166,198"
66,"1"
67,"1"
68,"170,74,107,127,225,169"
69,"1"
70,"1"
71,"1"
72,"1"
73,"1"
7

In [38]:
from neo4j import GraphDatabase
from rapidfuzz import process, fuzz

def run_query(query):
    """Esegue una query su Neo4j e restituisce i risultati."""
    with driver.session() as session:
        return session.run(query).data()

# 1️⃣ Controllo delle tecniche con nomi simili
query_techniques = "MATCH (t:Tecnica) RETURN t.nome"
results_techniques = run_query(query_techniques)
tech_names = [res['t.nome'] for res in results_techniques]

search_term_technique = "Cottura Sottovuoto Multirealità Collassante"
tech_matches = process.extract(search_term_technique, tech_names, limit=5)

print("\n🔍 Tecniche con nomi simili:")
for match in tech_matches:
    print(f"- {match[0]} (similarità: {match[1]})")

# 2️⃣ Controllo delle licenze con nomi simili a "Luce" e Livello >= 3
query_licenses = "MATCH (l:Licenza) WHERE l.Livello >= 3 RETURN l.nome, l.Livello"
results_licenses = run_query(query_licenses)
license_names = [res['l.nome'] for res in results_licenses]

# Estraiamo anche i nomi simili alle licenze usando Jaro-Winkler
search_term_license = "Luce"
license_matches = process.extract(search_term_license, license_names, limit=5, scorer=fuzz.ratio)

print("\n🔍 Licenze con nomi simili a 'Luce' e livello >= 3:")
for match in license_matches:
    print(f"- {match[0]} (Livello: {match[1]})")

# 3️⃣ Costruzione di una query più flessibile
# Tecniche simili
tech_names_similar = [match[0] for match in tech_matches]
# Licenze simili
license_names_similar = [match[0] for match in license_matches]

# Creiamo una query con i risultati ottenuti
tech_condition = " OR ".join([f"t.nome CONTAINS '{name}'" for name in tech_names_similar])
license_condition = " OR ".join([f"l.nome CONTAINS '{name}'" for name in license_names_similar])

# Costruzione della query
query_flexible = f"""
MATCH (r:Ristorante)-[:OFFERTA]->(p:Piatto)-[:PREPARATO]->(t:Tecnica),
      (r)-[:HA]->(l:Licenza)
WHERE ({license_condition}) AND l.Livello >= 3 AND ({tech_condition})
RETURN p.numero, p.nome, t.nome, l.nome, l.Livello
ORDER BY p.numero
"""

print("\n🔍 Query riformulata:")
print(query_flexible)

# Eseguiamo la query per ottenere i risultati
results_flexible = run_query(query_flexible)
print("\n🔍 Risultati della query riformulata:")
for res in results_flexible:
    print(res)

# Chiudiamo la connessione al database
driver.close()



🔍 Tecniche con nomi simili:
- Cottura Sottovuoto Multirealità Collassante (similarità: 100.0)
- CotturaSottovuotoMultirealitàCollassante (similarità: 96.3855421686747)
- Sottovuoto (similarità: 90.0)
- Sottovuoto Antimateria (similarità: 85.5)
- Sottovuoto Bioma sintetico (similarità: 85.5)

🔍 Licenze con nomi simili a 'Luce' e livello >= 3:
- Quantica (Livello: 33.333333333333336)
- LTK (Livello: 28.57142857142857)
- Quantistica (Livello: 26.66666666666667)
- Quantistica15 (Livello: 23.529411764705888)
- Psionica (Livello: 16.666666666666664)

🔍 Query riformulata:

MATCH (r:Ristorante)-[:OFFERTA]->(p:Piatto)-[:PREPARATO]->(t:Tecnica),
      (r)-[:HA]->(l:Licenza)
WHERE (l.nome CONTAINS 'Quantica' OR l.nome CONTAINS 'LTK' OR l.nome CONTAINS 'Quantistica' OR l.nome CONTAINS 'Quantistica15' OR l.nome CONTAINS 'Psionica') AND l.Livello >= 3 AND (t.nome CONTAINS 'Cottura Sottovuoto Multirealità Collassante' OR t.nome CONTAINS 'CotturaSottovuotoMultirealitàCollassante' OR t.nome CONTAINS '

In [44]:
import json

def run_query(query):
    """Esegue una query su Neo4j e restituisce i risultati."""
    with driver.session() as session:
        return session.run(query).data()

def get_standardized_data_from_llm(data, llm):
    """Usa OpenAI per ottenere una versione standardizzata e comprensiva dei dati."""
    # Costruisci un prompt che includa l'intero nodo
    prompt = f"""
    Riformula questi dati in modo che siano coerenti e uniformi, senza perdere nessuna informazione originale. Ecco i dati: {json.dumps(data)}
    
    Restituisci SOLO il JSON valido e completo, senza formattazione markdown o decoratori.
    Il formato deve essere:
    [
      {{'nome' : 'valore', "parametro1": 'valore', "parametro2": 'valore', ... , "parametrox": 'valore'}},   
    ]  
    """

    # Chiamata al modello LLM per ottenere la valorizzazione delle relazioni
    response = llm.invoke(prompt, temperature=1.0, seed=42)
    content = response.content.strip()
    
    # Pulizia del JSON per rimuovere markup indesiderato
    if content.startswith('```') and content.endswith('```'):
        content = '\n'.join(content.split('\n')[1:-1])
    if content.startswith('```json'):
        content = content[7:].strip()
    elif content.startswith('json'):
        content = content[4:].strip()

    # Tentativo di parsing del JSON
    try:
        relazioni = json.loads(content)
    except json.JSONDecodeError as e:
        print(f"Errore nel parsing JSON delle relazioni: {str(e)}")
        try:
            relazioni = json.loads(repair_json(content))  # Chiamata di una funzione di riparazione JSON, se disponibile
        except Exception:
            # Fallback con richiesta di correzione all'LLM
            prompt_repair = f"Correggi questo JSON per Neo4j e assicurati che sia valido e completo:\n{content}"
            repaired = llm.invoke(prompt_repair).content.strip()
            
            # Pulizia del JSON riparato
            if repaired.startswith('```') and repaired.endswith('```'):
                repaired = '\n'.join(repaired.split('\n')[1:-1])
            if repaired.startswith('```json'):
                repaired = repaired[7:].strip()
            elif repaired.startswith('json'):
                repaired = repaired[4:].strip()
                
            try:
                relazioni = json.loads(repaired)
            except:
                print("Impossibile riparare il JSON delle relazioni.")
                relazioni = []

    # Assicurati che il risultato sia una lista
    if not isinstance(relazioni, list):
        relazioni = [relazioni]

    return relazioni

def extract_and_standardize_licenses(llm):
    """Estrai e standardizza le licenze utilizzando OpenAI."""
    # 1️⃣ Estrai tutte le licenze da Neo4j
    query_licenses = "MATCH (l:Licenza) RETURN l"
    results_licenses = run_query(query_licenses)
    
    # 2️⃣ Standardizza ogni licenza usando LLM
    standardized_licenses = []
    
    for res in results_licenses:
        # Estrai il nodo completo (licenza con tutte le proprietà)
        license_data = dict(res['l'])  # Converti il nodo in un dizionario
        
        # Usa OpenAI per ottenere una versione standardizzata del nodo
        standardized_license = get_standardized_data_from_llm(license_data, llm)
        
        # Aggiungi la licenza standardizzata alla lista
        standardized_licenses.append({
            'licenza_standardizzata': standardized_license,
            'licenza_originale': license_data
        })
    
    # 3️⃣ Restituisci le licenze standardizzate
    return standardized_licenses

# Esegui il processo di estrazione e standardizzazione
standardized_licenses = extract_and_standardize_licenses(llm)

# Visualizza i risultati
print("\n🔍 Licenze Standardizzate:")
for license in standardized_licenses:
    print(json.dumps(license, indent=2))

# Chiudiamo la connessione al database
driver.close()


C:\Users\lparigi\AppData\Local\Temp\ipykernel_4868\3671522909.py:5: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Errore nel parsing JSON delle relazioni: Extra data: line 4 column 1 (char 93)
Impossibile riparare il JSON delle relazioni.
Errore nel parsing JSON delle relazioni: Expecting property name enclosed in double quotes: line 1 column 3 (char 2)
Impossibile riparare il JSON delle relazioni.
Errore nel parsing JSON delle relazioni: Expecting property name enclosed in double quotes: line 2 column 4 (char 5)
Impossibile riparare il JSON delle relazioni.

🔍 Licenze Standardizzate:
{
  "licenza_standardizzata": [
    {
      "Livello": 3,
      "nome": "Gradi_influenza_tecnologico"
    }
  ],
  "licenza_originale": {
    "Livello": 3,
    "nome": "Gradi_influenza_tecnologico"
  }
}
{
  "licenza_standardizzata": [
    {
      "anno": 2,
      "nome": "Psionica IV"
    }
  ],
  "licenza_originale": {
    "anno": 2,
    "nome": "Psionica IV"
  }
}
{
  "licenza_standardizzata": [
    {
      "anno": 3,
      "nome": "Temporale II"
    }
  ],
  "licenza_originale": {
    "anno": 3,
    "nome": "Temp

In [42]:
# Funzione per serializzare l'output del LLM
def custom_serializer(obj):
    """
    Funzione di serializzazione personalizzata.
    Estrae solo le informazioni utili dall'oggetto 'AIMessage'.
    """
    if isinstance(obj, dict):  # Se l'oggetto è un dizionario, ritorna il dizionario
        return {key: custom_serializer(value) for key, value in obj.items()}
    elif isinstance(obj, list):  # Se l'oggetto è una lista, ritorna la lista
        return [custom_serializer(item) for item in obj]
    elif hasattr(obj, '__dict__'):  # Se è un oggetto con __dict__ (es. una classe)
        return custom_serializer(vars(obj))  # Converte gli attributi dell'oggetto in dizionario
    else:
        return obj  # Ritorna l'oggetto se è già serializzabile (come stringhe, numeri, ecc.)
    


print("\n🔍 Licenze Standardizzate:")
for license in standardized_licenses:
    # A questo punto, possiamo usare custom_serializer per serializzare i dati
    try:
        json_output = json.dumps(license, indent=2, default=custom_serializer)
        print(json_output)
    except Exception as e:
        print(f"Errore nella serializzazione: {e}")



🔍 Licenze Standardizzate:
{
  "licenza_standardizzata": {
    "content": "Certamente! Ecco i dati riformulati in modo coerente e uniforme, mantenendo tutte le informazioni originali:\n\n```json\n{\n  \"Livello\": 3,\n  \"Nome\": \"Gradi Influenza Tecnologico\"\n}\n```\n\n**Modifiche apportate:**\n- **Uniformit\u00e0 nella formattazione dei campi:** Ho capitalizzato entrambe le chiavi (`Livello` e `Nome`) per mantenere una coerenza.\n- **Rimozione degli underscore:** Ho sostituito gli underscore con spazi nel valore di `\"Nome\"` per migliorare la leggibilit\u00e0.\n\nSe preferisci mantenere gli underscore o utilizzare un altro formato, fammelo sapere e posso adattare ulteriormente i dati secondo le tue esigenze!",
    "additional_kwargs": {
      "refusal": null
    },
    "response_metadata": {
      "token_usage": {
        "completion_tokens": 744,
        "prompt_tokens": 55,
        "total_tokens": 799,
        "completion_tokens_details": {
          "accepted_prediction_tokens"

In [28]:
for tipo, nome in entita_cercate.items():
    print(tipo)
    print(nome)

Tecnica
Fermentazione
Ingrediente
Erba Pipa
